# 01. Importing libraries

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

# 02. Importing and analyzing data

In [2]:
# Define path
path = r'/Users/elisabetta/Documents/12-2022 Instacart Basket Analysis'

In [3]:
df_ords_prods_merged_2 = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_2.pkl'))

In [4]:
# Create subset not to run into memory issues
df = df_ords_prods_merged_2[:1000000]

In [5]:
# Check output
df.head()

,order_id,user_id,number_of_orders,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,price,price_range,busy_day,busy_day_2,busiest_period_of_day
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,Medium,Regularly busy,Regularly busy,Average orders
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,Medium,Regularly busy,Least busy,Average orders
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,Medium,Regularly busy,Least busy,Most orders
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,Medium,Least busy,Least busy,Average orders
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,Medium,Least busy,Least busy,Most orders


In [6]:
# Check output
df.shape

(1000000, 17)

In [7]:
# Group by product_name
df.groupby('product_name')

In [8]:
# Group by department_id
df.groupby('department_id')

In [9]:
# Aggregate using number_of_orders. This calculates the mean of the number of orders per department
df.groupby('department_id').agg({'number_of_orders' : ['mean']})

,number_of_orders
,mean
department_id,
4,18.825780
7,17.472355
13,17.993423
14,19.246334
16,19.463012
17,11.294069
19,19.305237
20,17.599636


In [10]:
# Alternative method to do same
df.groupby('department_id')['number_of_orders'].mean()

department_id
4     18.825780
7     17.472355
13    17.993423
14    19.246334
16    19.463012
17    11.294069
19    19.305237
20    17.599636
Name: number_of_orders, dtype: float64

In [11]:
# Multiple aggregations
df.groupby('department_id').agg({'number_of_orders' : ['mean', 'min', 'max']})

number_of_orders        
                          mean min max
department_id                         
4                    18.825780   1  99
7                    17.472355   1  99
13                   17.993423   1  99
14                   19.246334   1  99
16                   19.463012   1  99
17                   11.294069   1  98
19                   19.305237   1  99
20                   17.599636   1  99

In [12]:
# Create column upon which flag will be based. Then assign it to customers WITHIN MAIN DATAFRAME df_ords_prods_merged_2

In [13]:
df_ords_prods_merged_2['max_number_of_orders'] = df_ords_prods_merged_2.groupby(['user_id'])['number_of_orders'].transform(np.max)

In [14]:
# Check output
df_ords_prods_merged_2.head()

,order_id,user_id,number_of_orders,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,price,price_range,busy_day,busy_day_2,busiest_period_of_day,max_number_of_orders
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,Medium,Regularly busy,Regularly busy,Average orders,10
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,Medium,Regularly busy,Least busy,Average orders,10
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,Medium,Regularly busy,Least busy,Most orders,10
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,Medium,Least busy,Least busy,Average orders,10
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,Medium,Least busy,Least busy,Most orders,10


In [15]:
# NOTE. Function to remove limit on .head()
# pd.options.display.max_rows = None

In [16]:
# Create and assign values to loyalty flag by creating conditions using .loc()
df_ords_prods_merged_2.loc[df_ords_prods_merged_2['max_number_of_orders'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [17]:
df_ords_prods_merged_2.loc[(df_ords_prods_merged_2['max_number_of_orders'] <= 40) & (df_ords_prods_merged_2['max_number_of_orders'] > 10) , 'loyalty_flag'] = 'Regular customer'

In [18]:
df_ords_prods_merged_2.loc[df_ords_prods_merged_2['max_number_of_orders'] <= 10, 'loyalty_flag'] = 'New customer'

In [19]:
# Check output
df_ords_prods_merged_2['loyalty_flag'].value_counts(dropna = False)

Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: loyalty_flag, dtype: int64

In [20]:
# Check output CONTINUED
df_ords_prods_merged_2['max_number_of_orders']

0           10
1           10
2           10
3           10
4           10
            ..
32404854    31
32404855    31
32404856     3
32404857     3
32404858    16
Name: max_number_of_orders, Length: 32404859, dtype: int64

In [21]:
df_ords_prods_merged_2[['user_id', 'loyalty_flag', 'max_number_of_orders']].head(60)

,user_id,loyalty_flag,max_number_of_orders
0,1,New customer,10
1,1,New customer,10
2,1,New customer,10
3,1,New customer,10
4,1,New customer,10
5,1,New customer,10
6,1,New customer,10
7,1,New customer,10
8,1,New customer,10
9,1,New customer,10


In [22]:
df_ords_prods_merged_2.head()

,order_id,user_id,number_of_orders,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,price,price_range,busy_day,busy_day_2,busiest_period_of_day,max_number_of_orders,loyalty_flag
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,Medium,Regularly busy,Regularly busy,Average orders,10,New customer
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,Medium,Regularly busy,Least busy,Average orders,10,New customer
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,Medium,Regularly busy,Least busy,Most orders,10,New customer
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,Medium,Least busy,Least busy,Average orders,10,New customer
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,Medium,Least busy,Least busy,Most orders,10,New customer


# 03. Task 4.8 starts here

In [23]:
# STEP 2 Calculate the mean of the number of orders per department in actual dataframe
df_ords_prods_merged_2.groupby('department_id')

In [24]:
df_ords_prods_merged_2.groupby('department_id').agg({'number_of_orders' : ['mean']})

,number_of_orders
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


In [25]:
# STEP 3 Analzye results and compare with subset results
# Here obviously all departments appear and also the means of departments are different (e.g. dept 4) because here all data is taken into account, not only a subset
# Most depts average between 15 and 17 orders with a couple of outliers, i.e. 10 (20 ords), 18 (19 ords), and 21 (22 ords)

In [26]:
# STEP 4 Already done above

In [27]:
# STEP 5 Determine spending stats per category of customer
# Loyal customers spend on average less than new and regular customers (10.4 vs 13.3 and 12.5 dollars). They hopefully compensate that by ordering more often
df_ords_prods_merged_2.groupby('loyalty_flag').agg({'price' : ['mean', 'min', 'max']})

price              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

In [28]:
# STEP 6 Create spending flag per user based on average price across all their orders using given criteria
# First calculate average spend

In [29]:
df_ords_prods_merged_2['avg_spend'] = df_ords_prods_merged_2.groupby(['user_id'])['price'].transform(np.mean)

In [30]:
# STEP 6 CONTINUED Create spending flag and assign values to it using .loc()

In [31]:
df_ords_prods_merged_2.loc[df_ords_prods_merged_2['avg_spend'] < 10, 'spend_flag'] = 'Low spender'

In [32]:
df_ords_prods_merged_2.loc[df_ords_prods_merged_2['avg_spend'] >= 10, 'spend_flag'] = 'High spender'

In [33]:
# Check output
df_ords_prods_merged_2['spend_flag'].value_counts(dropna = False)

Low spender     31770614
High spender      634245
Name: spend_flag, dtype: int64

In [34]:
df_ords_prods_merged_2.head()

,order_id,user_id,number_of_orders,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,department_id,price,price_range,busy_day,busy_day_2,busiest_period_of_day,max_number_of_orders,loyalty_flag,avg_spend,spend_flag
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,7,9.0,Medium,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,7,9.0,Medium,Regularly busy,Least busy,Average orders,10,New customer,6.367797,Low spender
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,7,9.0,Medium,Regularly busy,Least busy,Most orders,10,New customer,6.367797,Low spender
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,7,9.0,Medium,Least busy,Least busy,Average orders,10,New customer,6.367797,Low spender
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,7,9.0,Medium,Least busy,Least busy,Most orders,10,New customer,6.367797,Low spender


In [35]:
# STEP 7 Create frequency flag according to given criteria
# First calculate median of days_since_prior_order
df_ords_prods_merged_2['median_d_s_p_o'] = df_ords_prods_merged_2.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [36]:
# STEP 7 CONTINUED Create frequency flag and assign values to it using .loc()

In [37]:
df_ords_prods_merged_2.loc[df_ords_prods_merged_2['median_d_s_p_o'] > 20, 'frequency_flag'] = 'Non-frequent customer'

In [38]:
df_ords_prods_merged_2.loc[(df_ords_prods_merged_2['median_d_s_p_o'] <= 20) & (df_ords_prods_merged_2['median_d_s_p_o'] > 10) , 'frequency_flag'] = 'Regular customer'

In [39]:
df_ords_prods_merged_2.loc[df_ords_prods_merged_2['median_d_s_p_o'] <= 10, 'frequency_flag'] = 'Frequent customer'

In [40]:
# Check output
df_ords_prods_merged_2['frequency_flag'].value_counts(dropna = False)

Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
NaN                             5
Name: frequency_flag, dtype: int64

In [41]:
# Check output continued
df_ords_prods_merged_2['median_d_s_p_o'].isna().sum()

5

In [42]:
# Check output. Why 5 values are NaN? Indeed, median is NULL
df_check = df_ords_prods_merged_2.loc[df_ords_prods_merged_2['median_d_s_p_o'].isna()]

In [43]:
df_check.head()

,order_id,user_id,number_of_orders,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,price_range,busy_day,busy_day_2,busiest_period_of_day,max_number_of_orders,loyalty_flag,avg_spend,spend_flag,median_d_s_p_o,frequency_flag
13645692,895835,159838,1,0,17,NaN,10749,3,0,Organic Red Bell Pepper,...,Medium,Busiest day,Busiest days,Average orders,1,New customer,7.42,Low spender,NaN,NaN
17251990,895835,159838,1,0,17,NaN,33401,6,0,Goat Cheese Crumbles,...,Medium,Busiest day,Busiest days,Average orders,1,New customer,7.42,Low spender,NaN,NaN
17622767,895835,159838,1,0,17,NaN,23695,2,0,California Veggie Burger,...,Low,Busiest day,Busiest days,Average orders,1,New customer,7.42,Low spender,NaN,NaN
24138593,895835,159838,1,0,17,NaN,21334,5,0,Organic Peeled Garlic,...,Medium,Busiest day,Busiest days,Average orders,1,New customer,7.42,Low spender,NaN,NaN
25880002,895835,159838,1,0,17,NaN,22198,1,0,4X Ultra Concentrated Natural Laundry Detergen...,...,Low,Busiest day,Busiest days,Average orders,1,New customer,7.42,Low spender,NaN,NaN


In [44]:
df_check.shape

(5, 23)

In [45]:
df_check_2 = df_ords_prods_merged_2.loc[df_ords_prods_merged_2['days_since_prior_order'].isna()]

In [46]:
df_check_2.shape

(2076096, 23)

In [47]:
df_check_2.head(30)

,order_id,user_id,number_of_orders,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,price_range,busy_day,busy_day_2,busiest_period_of_day,max_number_of_orders,loyalty_flag,avg_spend,spend_flag,median_d_s_p_o,frequency_flag
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Medium,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
23,2497897,52,1,1,9,NaN,196,4,0,Soda,...,Medium,Regularly busy,Busiest days,Most orders,27,Regular customer,6.356287,Low spender,8.0,Frequent customer
37,3205960,67,1,3,11,NaN,196,1,0,Soda,...,Medium,Regularly busy,Least busy,Most orders,24,Regular customer,7.656790,Low spender,7.0,Frequent customer
58,1889835,82,1,3,15,NaN,196,6,0,Soda,...,Medium,Regularly busy,Least busy,Most orders,19,Regular customer,6.441221,Low spender,11.5,Regular customer
75,520620,120,1,3,11,NaN,196,2,0,Soda,...,Medium,Regularly busy,Least busy,Most orders,3,New customer,9.385714,Low spender,19.0,Regular customer
78,2040988,195,1,1,14,NaN,196,2,0,Soda,...,Medium,Regularly busy,Busiest days,Most orders,58,Loyal customer,7.149730,Low spender,4.0,Frequent customer
95,3226575,360,1,5,12,NaN,196,1,0,Soda,...,Medium,Regularly busy,Regularly busy,Most orders,3,New customer,10.006250,High spender,4.0,Frequent customer
130,3371566,760,1,2,11,NaN,196,2,0,Soda,...,Medium,Regularly busy,Regularly busy,Most orders,4,New customer,4.550000,Low spender,10.0,Frequent customer
132,231498,793,1,1,10,NaN,196,1,0,Soda,...,Medium,Regularly busy,Busiest days,Most orders,5,New customer,6.030769,Low spender,21.0,Non-frequent customer
140,2696986,991,1,3,16,NaN,196,1,0,Soda,...,Medium,Regularly busy,Least busy,Most orders,5,New customer,6.590909,Low spender,7.0,Frequent customer


In [51]:
# Unable to determine why 5 values are NaN. Since it's a minuscule amount will just ignore them
# First Check shape of dataframe

In [52]:
df_ords_prods_merged_2.shape

(32404859, 23)

In [53]:
# Drop all rows where the value in the specified column(s) are NaNs
df_ords_prods_merged_2 = df_ords_prods_merged_2.dropna(subset=['median_d_s_p_o', 'frequency_flag'], how = 'all')

In [54]:
# Check output
df_ords_prods_merged_2.shape

(32404854, 23)

In [55]:
# Check again if column frequency_flag has ok values
df_ords_prods_merged_2['frequency_flag'].value_counts(dropna = False)

Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
Name: frequency_flag, dtype: int64

In [56]:
# STEP 9 Export to pickle
df_ords_prods_merged_2.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_merged_3.pkl'))